Importamos librerias

In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn.model_selection import cross_val_score
from sklearn import metrics

Función de One Hot Enconding

In [57]:
# Archivo utilizado para crear la función de One Hot Encoding
############################################################# 

# Función auxiliar: Toma un df de variables categoricas y selección el top_x de categorias
def one_hot_top_x(cat_df, variable,top_x_labels):
    for label in top_x_labels:
        cat_df[str(variable)+'_'+str(label)] = np.where(cat_df[variable]==label,1,0)

# Función One Hot Encoder: Toma un DF y le hace una transformación de One Hot Encoder para con las categorias más repetidas 
def one_hot_encoder(df,top_x):
    # Separo variables categoricas de númericas
    num_cols = df.select_dtypes(include=['float64', 'int']).columns.to_list()
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.to_list()
    
    # Armo un DF solo con variables categoricas
    cat_df = df.drop(num_cols,axis=1)
    # Armo un DF solo con variables categoricas
    num_df = df.drop(cat_cols,axis=1)
   
    # Itero entre las variables categoricas y me quedo con el top 10 variables
    for variable in cat_cols:
        top_10 = [x for x in cat_df[variable].value_counts().sort_values(ascending=False).head(top_x).index]
        one_hot_top_x(cat_df,variable,top_10)
    
    cat_df = cat_df.drop(cat_cols,axis=1)
    df = pd.concat([cat_df,num_df], axis =1)
    
    return df

Importamos datos

In [86]:
# Levantamos los datos procesados
datos_meli = pd.read_excel('data_ready_to_model_api_simple.xlsx')
datos_meli['type'] = 'meli'

# Uno los DF
frames = [datos_meli, datos_privados]
datos = pd.concat(frames)

# Los paso por One Hot Encoder
datos = one_hot_encoder(datos,100)

# Separo los datos de Meli
datos_meli = datos[(datos['type_meli'] == 1)] 
datos_meli = datos_meli.drop('type_meli',axis=1)

C:\Users\LUCIA~1.LOP\AppData\Local\Temp/ipykernel_19604/187505526.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  cat_df[str(variable)+'_'+str(label)] = np.where(cat_df[variable]==label,1,0)


MODELO DE REGRESION - Base

In [59]:
x = datos_meli.drop('price',axis=1)

#separte the predicting attribute into Y for model training 
y = datos_meli['price']

X_train, X_test, y_train, y_test = train_test_split(
                                        x,
                                        y,
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )


print("Partición de entrenamiento")
print("-----------------------")
print(y_train.describe())

print("Partición de test")
print("-----------------------")
print(y_test.describe())

Partición de entrenamiento
-----------------------
count    3.957900e+04
mean     2.876519e+06
std      2.615258e+06
min      1.000000e+05
25%      1.550000e+06
50%      2.350000e+06
75%      3.590000e+06
max      1.300000e+08
Name: price, dtype: float64
Partición de test
-----------------------
count    9.895000e+03
mean     2.933694e+06
std      3.277476e+06
min      1.000000e+05
25%      1.550000e+06
50%      2.360010e+06
75%      3.580000e+06
max      1.500000e+08
Name: price, dtype: float64


Entrenamos el modelo

In [60]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Miramos resultados

In [61]:
print('Mean Absolute Error:', round(metrics.mean_absolute_error(y_test, y_pred)/1000,2))
print('Mean Squared Error:', round(metrics.mean_squared_error(y_test, y_pred)/1000,2))
print('Root Mean Squared Error:', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred))/1000,2))

Mean Absolute Error: 667.89
Mean Squared Error: 7512690477.07
Root Mean Squared Error: 2740.93


MODELO DE REGRESION - Logaritmo de Precio (Logaritmo)

In [62]:
# Separamos en Train & Test

y_ln = np.log(datos_meli['price'])
X_train, X_test, y_train_ln, y_test_ln = train_test_split(
                                        x,
                                        y_ln,
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

# Entrenamos el modelo

model_ln = LinearRegression()
model_ln.fit(X_train, y_train_ln)
y_pred_ln = model_ln.predict(X_test)

#  Saco exponencial de los valores de Y
y_test = np.exp(y_test_ln)
y_pred = np.exp(y_pred_ln)

print('Mean Absolute Error:', round(metrics.mean_absolute_error(y_test, y_pred)/1000,2))
print('Mean Squared Error:', round(metrics.mean_squared_error(y_test, y_pred)/1000,2))
print('Root Mean Squared Error:', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred))/1000,2))

Mean Absolute Error: 494.89
Mean Squared Error: 7094296315.81
Root Mean Squared Error: 2663.51
